In [0]:
import numpy as np
import pandas as pd
import os
import spacy
import en_core_web_sm
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS
import string
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,HashingVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
!pip install vaderSentiment
import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# preprocessing the data file
#  read the data
df1 = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/drugsComTrain_raw.csv")
df2 = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/drugsComTest_raw.csv")
# combine two file
df = pd.concat([df1, df2])
df
# rename the cols
df.columns = ['ID','drug name','condition','review','rating','date','useful count']

In [0]:
df2 = df[df['useful count'] > 10]  

In [0]:
df_condition = df2.groupby(['condition'])['drug name'].nunique().sort_values(ascending=False)
df_condition = pd.DataFrame(df_condition).reset_index()
df_condition.tail(20)

,condition,drug name
706,Rejection Prophylaxis,1
707,Ectopic Pregnancy,1
708,"Skin Disinfection, Preoperative",1
709,Muscle Twitching,1
710,Post-Cholecystectomy Diarrhea,1
711,Lewy Body Dementia,1
712,Short Stature for Age,1
713,Meningitis,1
714,ailure to Thrive,1
715,Short Stature,1


In [0]:
df_condition_1 = df_condition[df_condition['drug name'] == 1].reset_index()

all_list = set(df.index)

# deleting them
condition_list = []
for i,j in enumerate(df['condition']):
    for c in list(df_condition_1['condition']):
        if j == c:
            condition_list.append(i)
            
new_idx = all_list.difference(set(condition_list))
df = df.iloc[list(new_idx)].reset_index()
del df['index']

In [0]:
df.shape

(160684, 7)

In [0]:
# removing the conditions with <span> in it.

all_list = set(df.index)
span_list = []
for i,j in enumerate(df['condition']):
    if "</span>" in str(j):
        span_list.append(i)
new_idx = all_list.difference(set(span_list))
df = df.iloc[list(new_idx)].reset_index()
del df['index']

In [0]:
import re
from bs4 import BeautifulSoup
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
# removing some stopwords from the list of stopwords as they are important for drug recommendation

stops = set(stopwords.words('english'))

not_stop = ["aren't","couldn't","didn't","doesn't","don't","hadn't","hasn't","haven't","isn't","mightn't",
            "mustn't","needn't","no","nor","not","shan't","shouldn't","wasn't","weren't","wouldn't"]
for i in not_stop:
    stops.remove(i)

In [0]:
stemmer = SnowballStemmer('english')

def review_to_words(raw_review):
    # 1. Delete HTML 
    review_text = BeautifulSoup(raw_review, 'html.parser').get_text()
    # 2. Make a space
    letters_only = re.sub('[^a-zA-Z]', ' ', review_text)
    # 3. lower letters
    words = letters_only.lower().split()
    # 5. Stopwords 
    meaningful_words = [w for w in words if not w in stops]
    # 6. Stemming
    stemming_words = [stemmer.stem(w) for w in meaningful_words]
    # 7. space join words
    return( ' '.join(stemming_words))

# create a list of stopwords
nlp = spacy.load('en_core_web_sm')
stop_words = spacy.lang.en.stop_words.STOP_WORDS
parser = English()
punctuations = string.punctuation
# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens


In [0]:
%time df['review_clean'] = df['review'].apply(review_to_words)
df.head()

CPU times: user 1min 26s, sys: 397 ms, total: 1min 26s
Wall time: 1min 26s


,ID,drug name,condition,review,rating,date,useful count,review_clean
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27,no side effect take combin bystol mg fish oil
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192,son halfway fourth week intuniv becam concern ...
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,use take anoth oral contracept pill cycl happi...
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10,first time use form birth control glad went pa...
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37,suboxon complet turn life around feel healthie...


In [0]:
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,2))
#  tf-idf vector
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [0]:
# part 1---vader sentiment analyzer for c_review
analyzer = SentimentIntensityAnalyzer()
# create new col vaderReviewScore based on C-review
df['vaderReviewScore'] = df['review_clean'].apply(lambda x: analyzer.polarity_scores(x)['compound'])

# define the positive, neutral and negative
positive_num = len(df[df['vaderReviewScore'] >=0.05])
neutral_num = len(df[(df['vaderReviewScore'] >-0.05) & (df['vaderReviewScore']<0.05)])
negative_num = len(df[df['vaderReviewScore']<=-0.05])

# create new col vaderSentiment based on vaderReviewScore
df['vaderSentiment'] = df['vaderReviewScore'].map(lambda x:int(2) if x>=0.05 else int(1) if x<=-0.05 else int(0) )
df['vaderSentiment'].value_counts() # 2-pos: 99519; 1-neg: 104434; 0-neu: 11110

# label pos/neg/neu based on vaderSentiment result
df.loc[df['vaderReviewScore'] >=0.05,"vaderSentimentLabel"] ="positive"
df.loc[(df['vaderReviewScore'] >-0.05) & (df['vaderReviewScore']<0.05),"vaderSentimentLabel"]= "neutral"
df.loc[df['vaderReviewScore']<=-0.05,"vaderSentimentLabel"] = "negative"

In [0]:
df['vaderReviewScore'].max()

0.992

In [0]:
df['vaderReviewScore'].min()

-0.9976

In [0]:
criteria = [df['vaderReviewScore'].between(-0.997, -0.799), df['vaderReviewScore'].between(-0.798, -0.601), df['vaderReviewScore'].between(-0.600, 0.403), df['vaderReviewScore'].between(-0.402, -0.205), df['vaderReviewScore'].between(-0.204, -0.007), df['vaderReviewScore'].between(-0.006,0.191), df['vaderReviewScore'].between(0.192, 0.389), df['vaderReviewScore'].between(0.390, 0.587), df['vaderReviewScore'].between(0.588, 0.785), df['vaderReviewScore'].between(0.786, 1)]
values = [1, 2, 3,4,5,6,7,8,9,10]

df['normalVaderScore'] = np.select(criteria, values, 0)

In [0]:
df

,ID,drug name,condition,review,rating,date,useful count,review_clean,vaderReviewScore,vaderSentiment,vaderSentimentLabel,normalVaderScore
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27,no side effect take combin bystol mg fish oil,-0.2960,1,negative,3
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192,son halfway fourth week intuniv becam concern ...,0.6929,2,positive,9
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,use take anoth oral contracept pill cycl happi...,0.2732,2,positive,3
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10,first time use form birth control glad went pa...,0.4199,2,positive,8
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37,suboxon complet turn life around feel healthie...,0.8934,2,positive,10
...,...,...,...,...,...,...,...,...,...,...,...,...
159803,191035,Campral,Alcohol Dependence,"""I wrote my first report in Mid-October of 201...",10,31-May-15,125,wrote first report mid octob not alcohol sinc ...,0.9362,2,positive,10
159804,127085,Metoclopramide,Nausea/Vomiting,"""I was given this in IV before surgey. I immed...",1,1-Nov-11,34,given iv surgey immedi becam anxious could not...,-0.2732,1,negative,3
159805,187382,Orencia,Rheumatoid Arthritis,"""Limited improvement after 4 months, developed...",2,15-Mar-14,35,limit improv month develop bad rash md refus c...,-0.7351,1,negative,2
159806,47128,Thyroid desiccated,Underactive Thyroid,"""I&#039;ve been on thyroid medication 49 years...",10,19-Sep-15,79,thyroid medic year spent first synthroid vario...,0.7655,2,positive,9


In [0]:
df['meanNormalizedScore'] = (df['rating'] + df['normalVaderScore'])/2
df.head()

,ID,drug name,condition,review,rating,date,useful count,review_clean,vaderReviewScore,vaderSentiment,vaderSentimentLabel,normalVaderScore,meanNormalizedScore
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27,no side effect take combin bystol mg fish oil,-0.2960,1,negative,3,6.0
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192,son halfway fourth week intuniv becam concern ...,0.6929,2,positive,9,8.5
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,use take anoth oral contracept pill cycl happi...,0.2732,2,positive,3,4.0
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10,first time use form birth control glad went pa...,0.4199,2,positive,8,8.0
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37,suboxon complet turn life around feel healthie...,0.8934,2,positive,10,9.5


In [0]:
grouped = df.groupby(['condition','drug name', 'ID']).agg({'meanNormalizedScore' : ['mean']})
grouped.to_csv('Medicare_Normalized_results')
grouped1 = grouped.reset_index()
grouped1.head(100)

,condition,drug name,ID,meanNormalizedScore
,,,,mean
0,ADHD,Adderall,88252,4.5
1,ADHD,Adderall,88254,6.0
2,ADHD,Adderall,88255,10.0
3,ADHD,Adderall,88256,8.5
4,ADHD,Adderall,88257,5.5
...,...,...,...,...
95,ADHD,Adderall,88415,9.5
96,ADHD,Adderall,88417,9.0
97,ADHD,Adderall,88419,8.5


In [0]:
grouped1.set_index('ID')

,condition,drug name,meanNormalizedScore
,,,mean
ID,,,
88252,ADHD,Adderall,4.5
88254,ADHD,Adderall,6.0
88255,ADHD,Adderall,10.0
88256,ADHD,Adderall,8.5
88257,ADHD,Adderall,5.5
...,...,...,...
87551,zen Shoulde,Naproxen,6.0
87689,zen Shoulde,Naproxen,3.0


In [0]:
user_ratings = grouped1.pivot_table(index = ['condition'], columns = ['ID'], values = 'meanNormalizedScore')
user_ratings1 = grouped1.pivot_table(index = ['condition'], columns = ['ID'], values = 'meanNormalizedScore')
user_ratings.head(20)
user_ratings.to_csv('Itemtoitem_recom.csv')

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [0]:
user_ratings.iloc[0,:].sum(axis=0)

4.889422200449189e-13

In [0]:
#Let's predict the rating for the first drug  for the 1st condition.
#Therefore, we need to find the similarity between the them
import math as np
def takesec(num):
    return num[1]

cosine_vector = []
num = 0
l1 = 0
l2 = 0
for i in range(1,user_ratings.shape[0]):
    num = 0
    l1 = 0
    l2 = 0
    for j in range(user_ratings.shape[1]):
        if not np.isnan(user_ratings.iloc[i,j]) and not np.isnan(user_ratings.iloc[0,j]):
            num = num + (user_ratings.iloc[i,j] * user_ratings.iloc[0,j])
        if not np.isnan(user_ratings.iloc[i,j]):
            l1 = l1 + (user_ratings.iloc[i,j] * user_ratings.iloc[i,j])
        if not np.isnan(user_ratings.iloc[0,j]):
            l2 = l2 + (user_ratings.iloc[0,j] * user_ratings.iloc[0,j])
    eventual_prod = np.sqrt(l1) * np.sqrt(l2)
    if eventual_prod != 0 :
        eventual_div = num/eventual_prod
        cosine_vector.append([i,eventual_div])
        cosine_vector.sort(key=takesec, reverse=True)

In [0]:
cosine_vector[:100000]
user_ratings1.iloc[4,1]

5.5

In [0]:
import csv
# opening the csv file in 'w+' mode 
file = open('cosine_vec.csv', 'w+', newline ='') 
  
# writing the data into the file 
with file:     
    write = csv.writer(file) 
    write.writerows(cosine_vector) 

In [0]:

#Let's consider the top 50 similar rated drugs and predict the output.

predict_drug = int(input ("Enter a drug ID within range  0 to 1000000 : "))
count = 0
num1 = 0
den1 = 0
for i in cosine_vector:
    if user_ratings1.iloc[i[0],predict_drug] > 0:
            count = count + 1
            num1 = num1 + i[1] * user_ratings1.iloc[i[0],predict_drug]
            #print(num1)
            den1 = den1 + i[1]
            #print(den1)
            #print("{}..{}".format(i[0],i[1]))
        
    if count == 50:
        print ("Reached 50 :)")
        break

print("Expected Rating for 1st Drug for condition: {} is :{}".format(predict_drug,num1/den1))

Enter a drug ID within range  0 to 1000000 : 61002
Expected Rating for 1st Drug for condition: 61002 is :nan


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in double_scalars


In [0]:
for i in range(user_ratings1.shape[1]):
    if user_ratings1.iloc[0,i] > 0:
        print("Id = {} , Rating = {}".format(i,user_ratings1.iloc[0,i]))

Id = 584 , Rating = 5.5
Id = 585 , Rating = 9.0
Id = 587 , Rating = 6.0
Id = 588 , Rating = 3.5
Id = 589 , Rating = 6.5
Id = 590 , Rating = 6.0
Id = 591 , Rating = 9.0
Id = 1858 , Rating = 9.0
Id = 1893 , Rating = 8.5
Id = 1894 , Rating = 10.0
Id = 1895 , Rating = 6.0
Id = 1896 , Rating = 3.5
Id = 1897 , Rating = 4.0
Id = 1898 , Rating = 2.5
Id = 1899 , Rating = 2.0
Id = 1900 , Rating = 1.0
Id = 1901 , Rating = 2.5
Id = 1902 , Rating = 1.5
Id = 9035 , Rating = 8.5
Id = 9036 , Rating = 3.0
Id = 9037 , Rating = 2.0
Id = 9038 , Rating = 10.0
Id = 9039 , Rating = 2.0
Id = 9040 , Rating = 8.0
Id = 9041 , Rating = 6.0
Id = 9042 , Rating = 9.5
Id = 9043 , Rating = 3.0
Id = 9044 , Rating = 8.5
Id = 9045 , Rating = 4.0
Id = 9046 , Rating = 8.0
Id = 9047 , Rating = 9.0
Id = 9048 , Rating = 2.5
Id = 9049 , Rating = 2.0
Id = 9050 , Rating = 2.0
Id = 9051 , Rating = 1.0
Id = 9052 , Rating = 1.5
Id = 9053 , Rating = 1.5
Id = 9054 , Rating = 9.0
Id = 9055 , Rating = 6.0
Id = 9056 , Rating = 9.0
Id = 